In [1]:
import numpy as np
import pandas as pd

In [2]:
import prepostprocessing.cleaning as cleaning
import prepostprocessing.pre_processing as preproc
from qapf import qapf
from qapf import cipw

___

# CIPWFULL normalized data

## Convert to CIPWFULL format

In [3]:
mineralogy_original = pd.read_excel("../_CIPW/CIPW/AREA4/interpolated_data.xlsx", index_col=0)
# Only iterested in sample number and oxides
mineralogy = mineralogy_original[['SiO2', 'TiO2', 'Al2O3', 'Fe2O3','FeO', 'MnO', 'MgO', 'CaO', 'Na2O', 'K2O', 'oth']]
mineralogy.columns

Index(['SiO2', 'TiO2', 'Al2O3', 'Fe2O3', 'FeO', 'MnO', 'MgO', 'CaO', 'Na2O',
       'K2O', 'oth'],
      dtype='object')

In [4]:
mineralogy = mineralogy.dropna()

In [5]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [6]:
# Remove 'l.i.'' and 'oth' columns and immediately normalize data
mineralogy = preproc.normalize(mineralogy.drop(["oth"], axis=1))

In [7]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [8]:
mineralogy.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
6686,68.934225,0.394434,16.125092,1.477166,2.592715,0.057146,1.563130,3.088643,3.608640,2.158807
7279,68.955963,0.393367,16.122151,1.473556,2.587797,0.056995,1.558732,3.081707,3.608819,2.160912
7280,68.948293,0.393719,16.123296,1.474721,2.589447,0.057035,1.560507,3.084403,3.608756,2.159823
7281,68.941395,0.394036,16.124316,1.475775,2.590937,0.057072,1.562088,3.086810,3.608699,2.158872
7282,68.935288,0.394320,16.125211,1.476715,2.592262,0.057105,1.563472,3.088921,3.608648,2.158059


In [10]:
mineralogy

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
6686,68.934225,0.394434,16.125092,1.477166,2.592715,0.057146,1.563130,3.088643,3.608640,2.158807
7279,68.955963,0.393367,16.122151,1.473556,2.587797,0.056995,1.558732,3.081707,3.608819,2.160912
7280,68.948293,0.393719,16.123296,1.474721,2.589447,0.057035,1.560507,3.084403,3.608756,2.159823
7281,68.941395,0.394036,16.124316,1.475775,2.590937,0.057072,1.562088,3.086810,3.608699,2.158872
7282,68.935288,0.394320,16.125211,1.476715,2.592262,0.057105,1.563472,3.088921,3.608648,2.158059
...,...,...,...,...,...,...,...,...,...,...
395789,69.910832,0.319820,15.363757,0.849546,1.985693,0.032998,0.887011,1.901745,3.166959,5.581640
396380,69.859140,0.323144,15.387334,0.850328,2.005541,0.033291,0.900785,1.923839,3.166854,5.549743
396381,69.862503,0.322922,15.385703,0.850308,2.004168,0.033271,0.899831,1.922312,3.166844,5.552138
396382,69.866524,0.322661,15.383840,0.850261,2.002590,0.033248,0.898738,1.920560,3.166845,5.554732


In [11]:
mineralogy1 = mineralogy.iloc[0:45000]
mineralogy2 = mineralogy.iloc[45000:90000]
mineralogy3 = mineralogy.iloc[90000:135000]
mineralogy4 = mineralogy.iloc[135000:180000]



In [12]:
cipw.convert_to_CIPWFULL_format(mineralogy1, "../_CIPW/CIPW/AREA4/area4_1.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy2, "../_CIPW/CIPW/AREA4/area4_2.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy3, "../_CIPW/CIPW/AREA4/area4_3.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy4, "../_CIPW/CIPW/AREA4/area4_4.txt", dataset_name="Vistelius (1995)")

_____

## Extract results from CIPWFULL results file

**I only selected quartz, plagioclase and alkali feldspar here as they are of interest to us for the QAPF classification**  
**Feel free to extract addiotnal minerals if they would be of any interest to you**

**Not sure what to do with albite in the final assignments as by the book it should be accounted for as an alkali feldspar and thus be included in the sum of plagioclase but then the resulting plagioclase contents seem quite low.   
This is something you might look into although the method used here seems to be correct according to te resulting QAPF diagram.**

--> indeed when reading the paper of Streckeisen (and other sources), albite should be accounted for as an alkali feldspar, also saw that the datapoints were generally located more to the right side (plagioclase), maybe an overestimation of plagioclase?

--> I changed the settings below so that albite belongs to the alkali feldspars, Think this figure should be more correct according to the papers, but the cotent of plagioclase is indeed quite low. 

In [14]:
df_final1 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA4/area4_1_results_excel.txt")
df_final2 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA4/area4_2_results_excel.txt")
df_final3 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA4/area4_3_results_excel.txt")
df_final4 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA4/area4_4_results_excel.txt")

In [15]:
df_final = pd.concat([df_final1, df_final2, df_final3, df_final4])

In [16]:
df_final

,Q,P,K
SAMPLE,,,
6686,29.367157,42.801843,15.810000
7279,29.403294,42.768836,15.822562
7280,29.391005,42.781725,15.816071
7281,29.379938,42.793223,15.810406
7282,29.370081,42.803310,15.805578
...,...,...,...
395789,23.599543,33.550560,35.662237
396380,23.588665,33.659308,35.473641
396381,23.588591,33.651640,35.487819


## QAPF classification

In [17]:
df_final["QAPF"] = qapf.check_QAPF_region(df_final)

In [18]:
df_final["QAPF"].unique()

array(['granodiorite', 'monzo granite', 'quartz monzonite'], dtype=object)

In [19]:
df_final.to_excel("../_CIPW/CIPW/AREA4/qapf.xlsx", index=True)

-------

In [20]:
df_final.iloc[:, 3].value_counts().to_excel("../_CIPW/CIPW/AREA4/QAPF_counts.xlsx")

-----

In [21]:
QAPF_control = pd.read_excel("../_CIPW/CIPW/1_normalized.xlsx", index_col=0)
Area = pd.read_excel("../_INTERPOLATION/area_subdivided.xlsx", index_col=[0], usecols = [0, 1, 2, 3, 4, 29])

In [22]:
QAPF_control = pd.concat([Area, QAPF_control], axis = 1)
QAPF_control = QAPF_control.dropna()
QAPF_control

,Y_UTM,X_UTM,ZoneNumber,ZoneLetter,area,Q,P,K,QAPF
1,-1.119047e+06,5.136464e+06,57.0,V,2.0,52.526539,17.979983,22.933498,monzo granite
2,-2.003327e+06,6.758106e+06,57.0,V,1.0,45.821201,19.425633,32.374005,monzo granite
3,3.254870e+05,6.945620e+06,57.0,V,3.0,50.716606,6.475010,37.375668,syeno granite
4,-1.851743e+05,6.896371e+06,57.0,V,3.0,41.397182,23.234657,27.592905,monzo granite
5,7.165453e+05,7.654590e+06,57.0,V,4.0,54.609970,18.013528,19.746868,monzo granite
...,...,...,...,...,...,...,...,...,...
4654,-1.218800e+06,5.397379e+06,57.0,V,2.0,0.000000,52.913423,17.411244,monzodiorite monzogabbro
4655,7.373749e+05,7.408189e+06,57.0,V,4.0,0.000000,59.865509,15.514612,monzodiorite monzogabbro
4656,-1.164355e+06,5.436888e+06,57.0,V,2.0,0.000000,52.355599,12.039131,monzodiorite monzogabbro
4658,-1.218014e+05,6.800765e+06,57.0,V,3.0,0.000000,71.436466,10.147088,monzodiorite monzogabbro


In [23]:
QAPF_control_area4 = QAPF_control[QAPF_control['area'] ==4]
QAPF_control_area4.to_excel("../_CIPW/CIPW/AREA4/QAPF_control_points.xlsx", index=True)

In [24]:
QAPF_control_area4

,Y_UTM,X_UTM,ZoneNumber,ZoneLetter,area,Q,P,K,QAPF
5,7.165453e+05,7.654590e+06,57.0,V,4.0,54.609970,18.013528,19.746868,monzo granite
21,1.317109e+06,7.163468e+06,57.0,V,4.0,42.794945,38.867715,9.810512,granodiorite
29,1.291319e+06,7.013925e+06,57.0,V,4.0,45.206795,45.731351,4.949613,tonalite
53,1.257902e+06,7.650810e+06,57.0,V,4.0,39.577846,26.649833,30.302172,monzo granite
62,1.365606e+06,7.599095e+06,57.0,V,4.0,36.875748,31.213181,28.889572,monzo granite
...,...,...,...,...,...,...,...,...,...
4635,8.124596e+05,7.045757e+06,57.0,V,4.0,9.264481,55.097463,10.498079,quartz monzodiorite\nquartz monzogabbro
4639,7.921554e+05,7.112601e+06,57.0,V,4.0,3.464872,56.770607,14.234790,monzodiorite monzogabbro
4651,6.209919e+05,7.621993e+06,57.0,V,4.0,1.172160,50.797393,14.178667,monzodiorite monzogabbro
4655,7.373749e+05,7.408189e+06,57.0,V,4.0,0.000000,59.865509,15.514612,monzodiorite monzogabbro
